In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation

if os.getcwd()[-7:] == 'AAM_Sim':
    print(os.getcwd())
else:
    path_parent = os.path.dirname(os.getcwd())
    os.chdir(path_parent)
    path_parent = os.path.dirname(os.getcwd())
    os.chdir(path_parent)
    print(os.getcwd())
    
from Engine.GlobalClock import GlobalClock, Agent
from Engine.State import State
from Engine.Aircraft import AircraftType
from Engine.FlightPlan import FlightPlan
from Engine.MultiRotorController import MultiRotorController
from Engine.Wind import WindField
from Engine.Sensors import NavUpdate, GPSPosNavUpdate
from Engine.DragModel import DragModel
from Engine.TrackingSystem import *
from Engine.ConflictDetector import ConflictDetector

from CrossPlatformDev import my_print, join_str

import time

C:\Users\quekz\PycharmProjects\AAM_Sim


In [2]:
def FP_gen(HDG, speed, time, num_wpts, ini_pos):
    """Function that generates a flight plan according to encounter design. 
    A/C have fixed heading and will fly toward WPTs. 
    HDG --> desired A/C HDG (counter-clockwise from x-axis in RADIANS)
    speed --> desired A/C cruise spd
    time --> approximate duration for each flight leg (between 2 consecutive wpts)
    num_wpts --> number of wpts along the flight path
    ini_pos --> initial position of the A/C
    """
    wpt_list = []
    wpt_times = []
    spds = []
    mode_list = []
    duration_list = []
    for i in range(num_wpts):
        wpt_list.append(ini_pos + i*speed*time*np.array([np.cos(HDG), np.sin(HDG), 0]))
        wpt_times.append(i*time*1.1) # Give a 10% margin to each leg duration
        if i > 0:
            spds.append(speed)
            mode_list.append('Direct_P2P')
            duration_list.append(time*1.1) # Give a 10% margin to each leg duration
    wpt_start_list = wpt_list[0:num_wpts-1]
    wpt_end_list = wpt_list[1:]
    wpt_start_time = wpt_times[0:num_wpts-1]
    wpt_end_time = wpt_times[1:]
    return (FlightPlan(spds, mode_list,
                       wpt_start_list, wpt_start_time,
                       wpt_end_list, wpt_end_time, duration_list), wpt_list)


def calc_diameter(mass):
    """Data for this function obtained from log-log plot of popular DJI quadrotors"""
    if isinstance(mass, type(np.array([]))):
        diameter = (np.random.uniform(0.8, 1.2, size=len(mass)) * np.exp((0.3878 * np.log(mass) + 2.1387))).round(1)
    else:
        diameter = round(np.random.uniform(0.8, 1.2) * np.exp((0.3878 * np.log(mass) + 2.1387)), 1)
    return diameter

In [3]:
# File to be used for generating initial params

# Number of runs
num_runs = 100000

# Global Params

GlobalPhysicsUpdateRate = 240 * np.ones(num_runs)
GlobalSimStopTime = 200 * np.ones(num_runs)
Cruise_Leg_Time = 15 * np.ones(num_runs)
Num_Legs = 5 * np.ones(num_runs)

WindUpdateRate = 1 * np.ones(num_runs)
Wind_auto_x = 0.9 * np.ones(num_runs)
Wind_auto_y = 0.9 * np.ones(num_runs)
Wind_auto_z = 0 * np.ones(num_runs)
Wind_std_x = 3 * np.ones(num_runs)
Wind_std_y = 3 * np.ones(num_runs)
Wind_std_z = 0 * np.ones(num_runs)
Wind_mean_x = 0 * np.ones(num_runs)
Wind_mean_y = 0 * np.ones(num_runs)
Wind_mean_z = 0 * np.ones(num_runs)

# Some shared AC Params for this simulation

# AC GPS Error Models
GPS_Pos_Error_Hor_Auto = 0.994
GPS_Pos_Error_Vert_Auto = 0.994
# GPS_Pos_Error_Hor_Std = 3.5 # (radial)
# GPS_Pos_Error_Hor_Std = (GPS_Pos_Error_Hor_Std)/np.sqrt(0.5*(4-np.pi)) # Check formula! 
GPS_Pos_Error_Hor_Std = 4.08538983
# GPS_Pos_Error_Hor_Std = (GPS_Pos_Error_Hor_Std/2)/np.sqrt(0.5*(4-np.pi)) # Pls chk formula before running!
# GPS_Pos_Error_Vert_Std = 3.5
GPS_Pos_Error_Vert_Std = 1.53064037

# AC Cruise Spds
upper_spd = 20
lower_spd = 5

# Tracking Params
TrackingUnit_UpdateRate = 2 
TrackingUnit_Pos_Quant = 0 # nearest meter accuracy
TrackingUnit_Vel_Quant = 1 # nearest decimeter/sec accuracy
TrackingUnit_Avail = 0.9 
Latency = 0.5


# AC1 Params
AC1_Mass = np.random.uniform(0.25, 25, size=num_runs)
AC1_Prop_Diameter = calc_diameter(AC1_Mass)
AC1_HDG = np.random.uniform(0, 360, size=num_runs)
AC1_HDG = AC1_HDG * np.pi/180
AC1_Cruise_Speed = np.random.uniform(lower_spd,upper_spd, size=(num_runs))
radii = np.random.uniform(20,50, size=(num_runs)) * AC1_Cruise_Speed
AC1_Ini_Pos_x = -radii * np.cos(AC1_HDG)
AC1_Ini_Pos_y = -radii * np.sin(AC1_HDG)
AC1_Ini_Pos_z =  np.zeros(num_runs)

AC1_Ini_Vel_x = np.zeros(num_runs)
AC1_Ini_Vel_y = np.zeros(num_runs)
AC1_Ini_Vel_z = np.zeros(num_runs)

AC1_GPS_horizontal_std = GPS_Pos_Error_Hor_Std
AC1_GPS_horizontal_auto = GPS_Pos_Error_Hor_Auto
AC1_GPS_horizontal_auto = AC1_GPS_horizontal_auto * np.ones(num_runs)

AC1_GPS_vert_std = GPS_Pos_Error_Vert_Std
AC1_GPS_vert_auto = GPS_Pos_Error_Vert_Auto
AC1_GPS_vert_auto = AC1_GPS_vert_auto * np.ones(num_runs)

AC1_Ini_Pos_Err_x = np.random.normal(0, AC1_GPS_horizontal_std, size=num_runs)
AC1_Ini_Pos_Err_y = np.random.normal(0, AC1_GPS_horizontal_std, size=num_runs)
AC1_Ini_Pos_Err_z =  np.random.normal(0, AC1_GPS_vert_std, size=num_runs)

AC1_GPS_horizontal_std = AC1_GPS_horizontal_std * np.ones(num_runs)
AC1_GPS_vert_std = AC1_GPS_vert_std * np.ones(num_runs)

AC1_Ini_Vel_Err_x = np.zeros(num_runs) 
AC1_Ini_Vel_Err_y = np.zeros(num_runs)
AC1_Ini_Vel_Err_z = np.zeros(num_runs)

AC1_Ini_RPY_r = np.zeros(num_runs)
AC1_Ini_RPY_p = np.zeros(num_runs)
AC1_Ini_RPY_y = np.zeros(num_runs)

AC1_PhysicsUpdateRate = GlobalPhysicsUpdateRate # Already converted to array!
AC1_Controller_UpdateRate = 120 * np.ones(num_runs)
AC1_Start_Time = 0 * np.ones(num_runs)

AC1_PosUpdateRate = 10 * np.ones(num_runs)
AC1_VelUpdateRate = 10 * np.ones(num_runs)
AC1_AccelUpdateRate = 120 * np.ones(num_runs)
AC1_AirSpdSensorUpdateRate = 120 * np.ones(num_runs)

# Initialize AC1 Tracking System

AC1_TrackingUnit_UpdateRate = TrackingUnit_UpdateRate * np.ones(num_runs)
AC1_TrackingUnit_Pos_Quant = TrackingUnit_Pos_Quant * np.ones(num_runs)
AC1_TrackingUnit_Vel_Quant = TrackingUnit_Vel_Quant * np.ones(num_runs)
AC1_TrackingUnit_Avail = TrackingUnit_Avail * np.ones(num_runs)
AC1_Latency = Latency * np.ones(num_runs)


# AC2 Params
AC2_Mass = np.random.uniform(0.25, 25, size=num_runs)
AC2_Prop_Diameter = calc_diameter(AC2_Mass)
AC2_HDG = np.random.uniform(0, 360, size=num_runs)
AC2_HDG = AC2_HDG * np.pi/180
AC2_Cruise_Speed = np.random.uniform(lower_spd,upper_spd, size=(num_runs))
radii = np.random.uniform(20,50, size=(num_runs)) * AC2_Cruise_Speed
AC2_Ini_Pos_x = -radii * np.cos(AC2_HDG)
AC2_Ini_Pos_y = -radii * np.sin(AC2_HDG)
AC2_Ini_Pos_z =  np.zeros(num_runs)

AC2_Ini_Vel_x = np.zeros(num_runs)
AC2_Ini_Vel_y = np.zeros(num_runs)
AC2_Ini_Vel_z = np.zeros(num_runs)

AC2_GPS_horizontal_std = GPS_Pos_Error_Hor_Std
AC2_GPS_horizontal_auto = GPS_Pos_Error_Hor_Auto
AC2_GPS_horizontal_auto = AC2_GPS_horizontal_auto * np.ones(num_runs)

AC2_GPS_vert_std = GPS_Pos_Error_Vert_Std
AC2_GPS_vert_auto = GPS_Pos_Error_Vert_Auto
AC2_GPS_vert_auto = AC2_GPS_vert_auto * np.ones(num_runs)

AC2_Ini_Pos_Err_x = np.random.normal(0, AC2_GPS_horizontal_std, size=num_runs)
AC2_Ini_Pos_Err_y = np.random.normal(0, AC2_GPS_horizontal_std, size=num_runs)
AC2_Ini_Pos_Err_z =  np.random.normal(0, AC2_GPS_vert_std, size=num_runs)

AC2_GPS_horizontal_std = AC2_GPS_horizontal_std * np.ones(num_runs)
AC2_GPS_vert_std = AC2_GPS_vert_std * np.ones(num_runs)

AC2_Ini_Vel_Err_x = np.zeros(num_runs) 
AC2_Ini_Vel_Err_y = np.zeros(num_runs)
AC2_Ini_Vel_Err_z = np.zeros(num_runs)

AC2_Ini_RPY_r = np.zeros(num_runs)
AC2_Ini_RPY_p = np.zeros(num_runs)
AC2_Ini_RPY_y = np.zeros(num_runs)

AC2_PhysicsUpdateRate = GlobalPhysicsUpdateRate # Already converted to array!
AC2_Controller_UpdateRate = 120 * np.ones(num_runs)
AC2_Start_Time = 0 * np.ones(num_runs)

AC2_PosUpdateRate = 10 * np.ones(num_runs)
AC2_VelUpdateRate = 10 * np.ones(num_runs)
AC2_AccelUpdateRate = 120 * np.ones(num_runs)
AC2_AirSpdSensorUpdateRate = 120 * np.ones(num_runs)

# Initialize AC2 Tracking System

AC2_TrackingUnit_UpdateRate = TrackingUnit_UpdateRate * np.ones(num_runs)
AC2_TrackingUnit_Pos_Quant = TrackingUnit_Pos_Quant * np.ones(num_runs)
AC2_TrackingUnit_Vel_Quant = TrackingUnit_Vel_Quant * np.ones(num_runs)
AC2_TrackingUnit_Avail = TrackingUnit_Avail * np.ones(num_runs)
AC2_Latency = Latency * np.ones(num_runs)

In [4]:
Run = np.arange(0, num_runs)

param_df = pd.DataFrame({'Run':Run, 
                         'GlobalPhysicsUpdateRate':GlobalPhysicsUpdateRate, 'GlobalSimStopTime':GlobalSimStopTime,
                         'Cruise_Leg_Time':Cruise_Leg_Time, 'Num_Legs':Num_Legs, 
                         'WindUpdateRate':WindUpdateRate,
                         'Wind_auto_x':Wind_auto_x, 'Wind_auto_y':Wind_auto_y, 'Wind_auto_z':Wind_auto_z,
                         'Wind_std_x':Wind_std_x, 'Wind_std_y':Wind_std_y, 'Wind_std_z':Wind_std_z,
                         'Wind_mean_x':Wind_mean_x, 'Wind_mean_y':Wind_mean_y, 'Wind_mean_z':Wind_mean_z,
                         # AC1 Params
                         'AC1_Prop_Diameter':AC1_Prop_Diameter, 'AC1_Mass':AC1_Mass,
                         'AC1_HDG':AC1_HDG,
                         'AC1_Cruise_Speed':AC1_Cruise_Speed,
                         'AC1_Ini_Pos_x':AC1_Ini_Pos_x, 'AC1_Ini_Pos_y':AC1_Ini_Pos_y, 'AC1_Ini_Pos_z':AC1_Ini_Pos_z,
                         'AC1_Ini_Vel_x':AC1_Ini_Vel_x, 'AC1_Ini_Vel_y':AC1_Ini_Vel_y, 'AC1_Ini_Vel_z':AC1_Ini_Vel_z,
                         'AC1_GPS_horizontal_std':AC1_GPS_horizontal_std, 
                         'AC1_GPS_horizontal_auto':AC1_GPS_horizontal_auto,
                         'AC1_GPS_vert_std':AC1_GPS_vert_std, 
                         'AC1_GPS_vert_auto':AC1_GPS_vert_auto,
                         'AC1_Ini_Pos_Err_x':AC1_Ini_Pos_Err_x, 
                         'AC1_Ini_Pos_Err_y':AC1_Ini_Pos_Err_y, 
                         'AC1_Ini_Pos_Err_z':AC1_Ini_Pos_Err_z,
                         'AC1_Ini_Vel_Err_x':AC1_Ini_Vel_Err_x, 
                         'AC1_Ini_Vel_Err_y':AC1_Ini_Vel_Err_y, 
                         'AC1_Ini_Vel_Err_z':AC1_Ini_Vel_Err_z, 
                         'AC1_Ini_RPY_r':AC1_Ini_RPY_r, 
                         'AC1_Ini_RPY_p':AC1_Ini_RPY_p, 
                         'AC1_Ini_RPY_y':AC1_Ini_RPY_y,
                         'AC1_PhysicsUpdateRate':AC1_PhysicsUpdateRate,
                         'AC1_Controller_UpdateRate':AC1_Controller_UpdateRate,
                         'AC1_Start_Time':AC1_Start_Time,
                         'AC1_PosUpdateRate':AC1_PosUpdateRate,
                         'AC1_VelUpdateRate':AC1_VelUpdateRate,
                         'AC1_AccelUpdateRate':AC1_AccelUpdateRate,
                         'AC1_AirSpdSensorUpdateRate':AC1_AirSpdSensorUpdateRate,
                         # AC1 Tracking System Params
                         'AC1_TrackingUnit_UpdateRate':AC1_TrackingUnit_UpdateRate,
                         'AC1_TrackingUnit_Pos_Quant':AC1_TrackingUnit_Pos_Quant,
                         'AC1_TrackingUnit_Vel_Quant':AC1_TrackingUnit_Vel_Quant,
                         'AC1_TrackingUnit_Avail':AC1_TrackingUnit_Avail,
                         'AC1_Latency':AC1_Latency,
                         # AC2 Params
                         'AC2_Prop_Diameter':AC2_Prop_Diameter, 'AC2_Mass':AC2_Mass,
                         'AC2_HDG':AC2_HDG,
                         'AC2_Cruise_Speed':AC2_Cruise_Speed,
                         'AC2_Ini_Pos_x':AC2_Ini_Pos_x, 'AC2_Ini_Pos_y':AC2_Ini_Pos_y, 'AC2_Ini_Pos_z':AC2_Ini_Pos_z,
                         'AC2_Ini_Vel_x':AC2_Ini_Vel_x, 'AC2_Ini_Vel_y':AC2_Ini_Vel_y, 'AC2_Ini_Vel_z':AC2_Ini_Vel_z,
                         'AC2_GPS_horizontal_std':AC2_GPS_horizontal_std, 
                         'AC2_GPS_horizontal_auto':AC2_GPS_horizontal_auto,
                         'AC2_GPS_vert_std':AC2_GPS_vert_std, 
                         'AC2_GPS_vert_auto':AC2_GPS_vert_auto,
                         'AC2_Ini_Pos_Err_x':AC2_Ini_Pos_Err_x, 
                         'AC2_Ini_Pos_Err_y':AC2_Ini_Pos_Err_y, 
                         'AC2_Ini_Pos_Err_z':AC2_Ini_Pos_Err_z,
                         'AC2_Ini_Vel_Err_x':AC2_Ini_Vel_Err_x, 
                         'AC2_Ini_Vel_Err_y':AC2_Ini_Vel_Err_y, 
                         'AC2_Ini_Vel_Err_z':AC2_Ini_Vel_Err_z, 
                         'AC2_Ini_RPY_r':AC2_Ini_RPY_r, 
                         'AC2_Ini_RPY_p':AC2_Ini_RPY_p, 
                         'AC2_Ini_RPY_y':AC2_Ini_RPY_y,
                         'AC2_PhysicsUpdateRate':AC2_PhysicsUpdateRate,
                         'AC2_Controller_UpdateRate':AC2_Controller_UpdateRate,
                         'AC2_Start_Time':AC2_Start_Time,
                         'AC2_PosUpdateRate':AC2_PosUpdateRate,
                         'AC2_VelUpdateRate':AC2_VelUpdateRate,
                         'AC2_AccelUpdateRate':AC2_AccelUpdateRate,
                         'AC2_AirSpdSensorUpdateRate':AC2_AirSpdSensorUpdateRate,
                         # AC2 Tracking System Params
                         'AC2_TrackingUnit_UpdateRate':AC2_TrackingUnit_UpdateRate,
                         'AC2_TrackingUnit_Pos_Quant':AC2_TrackingUnit_Pos_Quant,
                         'AC2_TrackingUnit_Vel_Quant':AC2_TrackingUnit_Vel_Quant,
                         'AC2_TrackingUnit_Avail':AC2_TrackingUnit_Avail,
                         'AC2_Latency':AC2_Latency
                         
 })

param_df

Run  GlobalPhysicsUpdateRate  GlobalSimStopTime  Cruise_Leg_Time  \
0          0                    240.0              200.0             15.0   
1          1                    240.0              200.0             15.0   
2          2                    240.0              200.0             15.0   
3          3                    240.0              200.0             15.0   
4          4                    240.0              200.0             15.0   
...      ...                      ...                ...              ...   
99995  99995                    240.0              200.0             15.0   
99996  99996                    240.0              200.0             15.0   
99997  99997                    240.0              200.0             15.0   
99998  99998                    240.0              200.0             15.0   
99999  99999                    240.0              200.0             15.0   

       Num_Legs  WindUpdateRate  Wind_auto_x  Wind_auto_y  Wind_auto_z  \
0           5.0             1.0          0.9          0.9          0.0   
1           5.0             1.0          0.9          0.9          0.0   
2           5.0             1.0          0.9          0.9          0.0   
3           5.0             1.0          0.9          0.9          0.0   
4           5.0             1.0          0.9          0.9          0.0   
...         ...             ...          ...          ...          ...   
99995       5.0             1.0          0.9          0.9          0.0   
99996       5.0             1.0          0.9          0.9          0.0   
99997       5.0             1.0          0.9          0.9          0.0   
99998       5.0             1.0          0.9          0.9          0.0   
99999       5.0             1.0          0.9          0.9          0.0   

       Wind_std_x  ...  AC2_Start_Time  AC2_PosUpdateRate  AC2_VelUpdateRate  \
0             3.0  ...             0.0               10.0               10.0   
1             3.0  ...             0.0               10.0               10.0   
2             3.0  ...             0.0               10.0               10.0   
3             3.0  ...             0.0               10.0               10.0   
4             3.0  ...             0.0               10.0               10.0   
...           ...  ...             ...                ...                ...   
99995         3.0  ...             0.0               10.0               10.0   
99996         3.0  ...             0.0               10.0               10.0   
99997         3.0  ...             0.0               10.0               10.0   
99998         3.0  ...             0.0               10.0               10.0   
99999         3.0  ...             0.0               10.0               10.0   

       AC2_AccelUpdateRate  AC2_AirSpdSensorUpdateRate  \
0                    120.0                       120.0   
1                    120.0                       120.0   
2                    120.0                       120.0   
3                    120.0                       120.0   
4                    120.0                       120.0   
...                    ...                         ...   
99995                120.0                       120.0   
99996                120.0                       120.0   
99997                120.0                       120.0   
99998                120.0                       120.0   
99999                120.0                       120.0   

       AC2_TrackingUnit_UpdateRate  AC2_TrackingUnit_Pos_Quant  \
0                              2.0                         0.0   
1                              2.0                         0.0   
2                              2.0                         0.0   
3                              2.0                         0.0   
4                              2.0                         0.0   
...                            ...                         ...   
99995                          2.0                         0.0   
99996    

In [5]:
# param_df['AC1_GPS_horizontal_auto']

In [6]:
param_df.columns

Index(['Run', 'GlobalPhysicsUpdateRate', 'GlobalSimStopTime',
       'Cruise_Leg_Time', 'Num_Legs', 'WindUpdateRate', 'Wind_auto_x',
       'Wind_auto_y', 'Wind_auto_z', 'Wind_std_x', 'Wind_std_y', 'Wind_std_z',
       'Wind_mean_x', 'Wind_mean_y', 'Wind_mean_z', 'AC1_Prop_Diameter',
       'AC1_Mass', 'AC1_HDG', 'AC1_Cruise_Speed', 'AC1_Ini_Pos_x',
       'AC1_Ini_Pos_y', 'AC1_Ini_Pos_z', 'AC1_Ini_Vel_x', 'AC1_Ini_Vel_y',
       'AC1_Ini_Vel_z', 'AC1_GPS_horizontal_std', 'AC1_GPS_horizontal_auto',
       'AC1_GPS_vert_std', 'AC1_GPS_vert_auto', 'AC1_Ini_Pos_Err_x',
       'AC1_Ini_Pos_Err_y', 'AC1_Ini_Pos_Err_z', 'AC1_Ini_Vel_Err_x',
       'AC1_Ini_Vel_Err_y', 'AC1_Ini_Vel_Err_z', 'AC1_Ini_RPY_r',
       'AC1_Ini_RPY_p', 'AC1_Ini_RPY_y', 'AC1_PhysicsUpdateRate',
       'AC1_Controller_UpdateRate', 'AC1_Start_Time', 'AC1_PosUpdateRate',
       'AC1_VelUpdateRate', 'AC1_AccelUpdateRate',
       'AC1_AirSpdSensorUpdateRate', 'AC1_TrackingUnit_UpdateRate',
       'AC1_TrackingUnit_Pos_Qu

In [7]:
# Plot some figures to check init settings

param_df['Relative Hdg (initial, degrees)'] = np.arccos(np.cos(param_df['AC1_HDG'])*np.cos(param_df['AC2_HDG']) \
                                                        + np.sin(param_df['AC1_HDG'])*np.sin(param_df['AC2_HDG']))*180/np.pi

In [8]:
fig, axs = plt.subplots(ncols=4, figsize=(16,5))

axs[0].hist(param_df['AC1_Cruise_Speed'], label='AC1', color='b', alpha=0.5)
axs[0].hist(param_df['AC2_Cruise_Speed'], label='AC2', color='orange', alpha=0.5)
axs[0].set_title('Cruise Spds')
axs[0].legend()

axs[1].hist(param_df['Relative Hdg (initial, degrees)'], color='b', alpha=0.5, label = 'Relative Hdg')
axs[1].set_title('Relative Hdg (initial, degrees)')
axs[1].legend()

axs[2].hist(param_df['AC1_Mass'], color='b', alpha=0.5, label = 'AC1')
axs[2].hist(param_df['AC2_Mass'], color='orange', alpha=0.5, label = 'AC2')
axs[2].set_title('Masses')
axs[2].legend()

axs[3].hist(param_df['AC1_Prop_Diameter'], color='b', alpha=0.5, label = 'AC1')
axs[3].hist(param_df['AC2_Prop_Diameter'], color='orange', alpha=0.5, label = 'AC2')
axs[3].set_title('Prop Diameters')
axs[3].legend()

plt.show()

In [9]:
# Save Initial Params
Init_Param_Path = join_str(os.getcwd(), 'MonteCarlo', 'TwoAircraftEncounter', 'Init_Param.csv')
Init_Param_Path

param_df.to_csv(Init_Param_Path)